In [1]:
import json
import os
import re
from time import sleep

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

In [3]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.get('https://quillbot.com/login?returnUrl=/summarize')

In [10]:
def login(email, password) -> None:
    email_field = driver.find_element(By.XPATH, '//*[@id="loginContainer"]/div/div[4]/div/div/input')
    password_field = driver.find_element(By.XPATH, '//*[@id="loginContainer"]/div/div[5]/div/div/input')

    email_field.send_keys(email)
    sleep(1)
    password_field.send_keys(password)
    ActionChains(driver).key_down(Keys.ENTER).key_up(Keys.ENTER).perform()

In [11]:
import SECRETS
login(SECRETS.EMAIL, SECRETS.PASSWORD)

In [12]:
FOLDER_PATH: str = 'C:\\Users\\Sharvin\\Downloads\\Documents\\Light Novels\\__Ongoing\\Rascal Does Not Dream Series\\Rascal Does Not Dream Series - Volume 07 [Yen Press][Kobo] (1)'

In [13]:
def make_chunks(text: str) -> list[list[str]]:
    """Split the text into 1,200 words chunks."""
    counter: int = 0
    chunk_length: int = 1200
    chunks: list[list[str]] = []
    sentences: list[str] = []
    for sentence in text.split('\n'):
        if len(sentence.split()) + counter < chunk_length:
            sentences.append(sentence)
            counter: int = counter + len(sentence.split())
        else:
            chunks.append(sentences[:])
            counter: int = 0
            sentences.clear()
            sentences.append(sentence)
            counter: int = counter + len(sentence.split())
    chunks.append(sentences[:])
    return chunks

In [14]:
def get_summary(text) -> str:
    textbox = driver.find_element(By.ID, 'inputBoxSummarizer')
    textbox.clear()
    sleep(1)
    textbox.send_keys(text)
    sleep(10)
    ActionChains(driver).key_down(Keys.CONTROL).key_down(Keys.ENTER).key_up(Keys.CONTROL).key_up(Keys.ENTER).perform()
    sleep(10)
    summary = driver.find_element(By.ID, 'outputBoxSummarizer')
    return summary.text

In [15]:
def save_to_file(text: str, file: str):
    file_part: str = 'summary' + ' - ' + file.title()
    SUMMARY: str = os.path.join(FOLDER_PATH, 'summary')
    file_summary: str = os.path.join(SUMMARY, file_part)
    if not os.path.exists(SUMMARY):
        os.mkdir(os.path.join(SUMMARY))
    with open(file_summary, 'a', encoding='utf-8') as f:
        f.write(text)
        f.write('\n\n')

In [16]:
for file in os.listdir(FOLDER_PATH):
    file_path: str = os.path.join(FOLDER_PATH, file)
    if not os.path.isfile(file_path):
        continue
    print(file)
    with open(file_path, 'r', encoding='utf-8') as f:
        text: str  = f.read()
        text: str = re.sub(r'\n{2,50}', '\n', text).strip()
        text: str = re.sub(r'(?<=\S)\?(?=\S)', ', ', text).strip()
        text: str = text.replace('—', ', ').strip()
        parts: list[list[str]] = make_chunks(text)
        for idx, part in enumerate(parts):
            print(f'{idx + 1}/{len(parts)}')
            text: str = '\n'.join(part)
            summary_text: str = get_summary(text)
            save_to_file(summary_text, file)
driver.quit()

02 - Epigraph.txt
1/1
03 - Chapter 1 A Gray and Desolate Landscape.txt
1/11
2/11
3/11
4/11
5/11


In [ ]:
print('COMPLETED')